~~~
Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-health/path-foundation/blob/master/notebooks/train_data_efficient_classifier_gcs.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgoogle-health%2Fpath-foundation%2Fmaster%2Fnotebooks%2Ftrain_data_efficient_classifier_gcs.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/path-foundation/blob/master/notebooks/train_data_efficient_classifier_gcs.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/google/path-foundation">
      <img alt="HuggingFace logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on HuggingFace
    </a>
  </td>
</tr></tbody></table>


# Train a Digital Pathology Linear Classifier From Images Stored on Google Cloud Storage (GCS)

This notebook is a demonstration of generating and using embeddings from the Path Foundation Serving API to train a linear classifier. This API enables users to compute embeddings for histopathology images. Note: This notebook is for API demonstration purposes only. As with all machine-learning use-cases it is critical to consider training and evaluation datasets that reflect the expected distribution of the intended use case.

**Additional details**: For this demo, patches sampled from whole slide images (WSIs) are downloaded from Google Cloud Storage. A subset of the patches will be sampled randomly from across all available slides and embeddings will be generated via the Path Foundation model. The Colab supports two modes of execution, embeddings can be generated locally or in the Cloud using a Pathology Embeddings Research Endpoint.

**Dataset**: This notebook uses the [CAMELYON16](https://camelyon16.grand-challenge.org/) dataset, which contains WSIs from lymph node specimens with and without metastatic breast cancer. Any work that uses this dataset should consider additional details along with usage and citation requirements listed on [their website](https://camelyon17.grand-challenge.org/Data/).

**Dataset citation**: Babak Ehteshami Bejnordi; Mitko Veta; Paul Johannes van Diest; Bram van Ginneken; Nico Karssemeijer; Geert Litjens; Jeroen A. W. M. van der Laak; and the CAMELYON16 Consortium. Diagnostic Assessment of Deep Learning Algorithms for Detection of Lymph Node Metastases in Women With Breast Cancer. JAMA. 2017;318(22):2199–2210. DOI: 10.1001/jama.2017.14585

In [ ]:
# @title Pip install EZ-WSI DICOMweb
%%capture
!pip install --upgrade ez_wsi_dicomweb>=6.0.8

### Define the endpoint to use to generate embeddings.

This Colab supports generating embeddings locally or from a Vertex AI hosted pathology embeddings endpoint. The endpoint selector defines the endpoint which will be used.



* **Generating Embeddings Locally** (Default): This option will generate embeddings using the pathology embeddings model hosted on [Hugging Face](https://huggingface.co/). Embeddings will be generated for the training and evaluation images hosted in GCS by downloading the images and running them across the model locally. The time required to complete this is typically 5 - 6 min but will be affected by the network bandwidth and computational resources of the local environment running the notebook. Retrieving the model from Hugging Face will require a valid Hugging Face account. Additional accounts are not required.
* **Generate Embeddings In the Cloud**: This option will generate embeddings in the Cloud using a Pathology Embeddings Endpoint. The parameters that follow the end point selector default to settings that will utilize the Google Research Pathology Embeddings Endpoint. These parameters can be changed to point to private deployments. There is no charge associated with using the Google Research Embeddings endpoint. However, access to it is restricted to gain please see the [Research Endpoint Access](https://developers.google.com/health-ai-developer-foundations/model-serving/research-endpoints) page for details.

In [ ]:
Endpoint = 'Generate Embeddings In the Cloud'  # @param ['Generate Embeddings Locally', 'Generate Embeddings In the Cloud']

Endpoint_Google_Cloud_Project = 'hai-cd3-foundations'  # @param {type: 'string'}
Endpoint_Google_Cloud_Location = 'us-central1'  # @param {type: 'string'}
Endpoint_ID = '162'  # @param {type: 'string'}

In [ ]:
# @title Retrieve list of images defined as representing cancer and benign imaging from Google Cloud Storage.
import google.cloud.storage

client = google.cloud.storage.Client.create_anonymous_client()
bucket = google.cloud.storage.Bucket(name='healthai-us', client=client)

# Patch imaging is statified by slide and stored within test and evaluation buckets
# Retrieve a list of imaging stored in buckets, image bytes not retrieved here.
cancer_imaging_training = list(
    bucket.list_blobs(prefix='pathology/training/cancer')
)
benign_imaging_training = list(
    bucket.list_blobs(prefix='pathology/training/benign')
)
cancer_imaging_eval = list(bucket.list_blobs(prefix='pathology/eval/cancer'))
benign_imaging_eval = list(bucket.list_blobs(prefix='pathology/eval/benign'))

In [ ]:
# @title Select a random subset of imaging for training and evaluation
import random

TRAINING_SIZE = 250
EVAL_SIZE = 65

# Randomize the image lists.
random.shuffle(cancer_imaging_training)
random.shuffle(benign_imaging_training)
random.shuffle(cancer_imaging_eval)
random.shuffle(benign_imaging_eval)

# Take the 250 examples for training
cancer_training = cancer_imaging_training[:TRAINING_SIZE]
benign_training = benign_imaging_training[:TRAINING_SIZE]

# Take the 65 examples for evaluation
cancer_eval = cancer_imaging_eval[:EVAL_SIZE]
benign_eval = benign_imaging_eval[:EVAL_SIZE]

In [ ]:
# @title Authenticate notebook.

from huggingface_hub import notebook_login
from huggingface_hub.utils import HfFolder
from google.colab import auth

if Endpoint == 'Generate Embeddings Locally':
  if 'hugging_face_login' not in globals():
    if HfFolder.get_token() is None:
        notebook_login()
    else:
      print("Token already set")
    global hugging_face_login
    hugging_face_login = True
elif 'gcp_user_auth' not in globals():
  # Authenticate user for access to Research Embedding Endpoint.
  # There will be a popup asking you to sign in with your user account
  # and approve access.
  auth.authenticate_user()
  global gcp_user_auth
  gcp_user_auth = True

In [ ]:
# @title Define Cloud or Local Endpoint used to Generate Embeddings.

import functools

from huggingface_hub import from_pretrained_keras
from ez_wsi_dicomweb import patch_embedding
from ez_wsi_dicomweb import patch_embedding_endpoints
import numpy as np
import tensorflow as tf


def _load_huggingface_model() -> tf.keras.Model:
  """Returns a model loaded from Hugging Face."""

  # Load model fom Hugging Face
  if 'loaded_model' not in globals():
    global loaded_model
    loaded_model = from_pretrained_keras(
        'google/path-foundation', compile=False
    )
  return loaded_model


def _endpoint_model(ml_model: tf.keras.Model, image: np.ndarray) -> np.ndarray:
  """Function ez-wsi will use to run local ML model."""
  result = ml_model.signatures['serving_default'](
      tf.cast(tf.constant(image), tf.float32)
  )
  return result['output_0'].numpy()


def create_embedding_endpoint_for_hugging_face_model() -> (
    patch_embedding_endpoints.LocalEndpoint
):
  """Returns a endpoint that generates embeddings using model run locally."""
  # Use functools to define the value passed to the first parameter of the
  # _endpoint model function
  endpoint_model = functools.partial(_endpoint_model, _load_huggingface_model())
  # Local endpoint takes a function which accepts a np.ndarray as a parameter
  # and returns the generated embeddings as a np.ndarray.
  # expected input shape [Batch, ImageDim, ImageDim, RGB_Channels]
  return patch_embedding_endpoints.LocalEndpoint(endpoint_model)

if Endpoint == 'Generate Embeddings Locally':
  endpoint = create_embedding_endpoint_for_hugging_face_model()
else:
  endpoint = patch_embedding_endpoints.V2PatchEmbeddingEndpoint(
      endpoint_api_version='v1',
      project_id=Endpoint_Google_Cloud_Project,
      endpoint_location=Endpoint_Google_Cloud_Location,
      endpoint_id=Endpoint_ID,
  )


In [ ]:
# @title Generate embeddings for pathology Embedding Endpoint
#
# Patch embeddings are computed here. This cell will take ~5 min to execute.
# Execution speed is dependent on local bandwith and compute. Imaging will be
# downloaded from cloud and embeddings will be generated locally using the
# model loaded from Hugging Face.

import concurrent.futures
from typing import List
from ez_wsi_dicomweb import credential_factory
from ez_wsi_dicomweb import gcs_image
from ez_wsi_dicomweb import patch_embedding_types



def generate_embeddings(patches) -> List[patch_embedding_types.EmbeddingResult]:
  """Returns embeddings for list (patches) of images using the endpoint defined."""
  # Performance tip: Defining the image dimensions improves performance by
  # enabling the client to know the dimensions of input imaging without having
  # to retrieve the imaging. The patch imaging used in this Colab was saved as
  # 224 x 224 pixels patches to match the endpoint input dimensions. If the
  # defined image dimension does not match the actual image dimension the
  # endpoint generating the image will resize the image to match the defined
  # dimensions.
  #
  # For this colab embeddings are generated from the whole image.
  embedding_result_iterator = patch_embedding.gcs_images_to_embeddings(
          endpoint,
          patches,
          image_dimensions=gcs_image.ImageDimensions(224, 224),
          credential_factory=credential_factory.NoAuthCredentialsFactory(),
      )
  return list(embedding_result_iterator)


# Requeset embeddings in parallel for all four datasets.
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
  results = list(
      executor.map(
          generate_embeddings,
          [cancer_training, benign_training, cancer_eval, benign_eval],
      )
  )
training_cancer_embeddings = results[0]
training_benign_embeddings = results[1]
eval_cancer_embeddings = results[2]
eval_benign_embeddings = results[3]


In [ ]:
# @title Organize embeddings for ML training
import os
from typing import Sequence, Tuple


def get_embeddings(
    embedding_results: Sequence[patch_embedding_types.EmbeddingResult],
) -> np.ndarray:
  """Returns numpy array of embeddings returned in embedding results list."""
  return np.array([e.embedding for e in embedding_results])


def get_slide_id(
    embedding_results: Sequence[patch_embedding_types.EmbeddingResult],
) -> List[str]:
  """Returns list of patch slide ids encoded into patch GCS file name.

  Patch file names were encoded with an id value that identifies the ID of the
  the slide they came from, extract IDs from file names to use a clustering
  index in StratifiedGroupKFold
  """
  slide_id = []
  # for each embedding result get the images GCS URI
  for uri in [e.patch.source.uri for e in embedding_results]:
    # split the file name into parts and extract the slide id
    filename = uri.split('/')[-1]
    slide_id.append(os.path.splitext(filename)[0].split('_')[-1])
  return slide_id


def concatenate_training_data_and_build_training_labels(
    cancer: Sequence[patch_embedding_types.EmbeddingResult],
    benign: Sequence[patch_embedding_types.EmbeddingResult],
) -> Tuple[np.ndarray, np.ndarray]:
  """Concatenate cancer and benign examples into and generate label data."""
  data = np.concatenate([get_embeddings(cancer), get_embeddings(benign)])
  labels = np.concatenate((np.ones(len(cancer)), np.zeros(len(benign))))
  return data, labels


# Embeddings and training lables
training_embeddings, training_labels = (
    concatenate_training_data_and_build_training_labels(
        training_cancer_embeddings, training_benign_embeddings
    )
)
# Slide Ids for clustering.
slide_ids = get_slide_id(training_cancer_embeddings)
slide_ids.extend(get_slide_id(training_benign_embeddings))

# Generate evaluation embeddings and labels
eval_embeddings, eval_labels = (
    concatenate_training_data_and_build_training_labels(
        eval_cancer_embeddings, eval_benign_embeddings
    )
)


In [ ]:
# @title Train a linear classifier using the embeddings
import warnings
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
import sklearn.preprocessing

with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  clf_pipeline = sklearn.pipeline.Pipeline([
      ('scaler', sklearn.preprocessing.StandardScaler()),
      (
          'logreg',
          sklearn.model_selection.GridSearchCV(
              sklearn.linear_model.LogisticRegression(
                  random_state=0,
                  multi_class='ovr',
                  verbose=False,
              ),
              cv=sklearn.model_selection.StratifiedGroupKFold(n_splits=5).split(
                  training_embeddings, y=training_labels, groups=slide_ids
              ),
              param_grid={'C': np.logspace(start=-4, stop=4, num=10, base=10)},
              scoring='roc_auc_ovr',
              refit=True,
          ),
      ),
  ]).fit(training_embeddings, training_labels)

  test_predictions = clf_pipeline.predict_proba(eval_embeddings)[:, 1]


In [ ]:
# @title Evaluate the linear classifiers performance using the eval patches
sklearn.metrics.roc_auc_score(eval_labels, test_predictions)

In [ ]:
# @title Plot the ROC Curve

display = sklearn.metrics.RocCurveDisplay.from_predictions(
    eval_labels, test_predictions, name="Tumor Classifier"
)
display.ax_.set_title("ROC of Tumor Classifier")

In [ ]:
# @title Find Youden's index for threshold selection

thresholds = np.linspace(0, 1, 100)
sensitivities = []
specificities = []
for threshold in thresholds:
  predictions = test_predictions > threshold
  sensitivities.append(sklearn.metrics.recall_score(eval_labels, predictions))
  specificities.append(
      sklearn.metrics.recall_score(eval_labels == 0, predictions == 0)
  )
index = np.argmax(np.array(sensitivities) + np.array(specificities))
best_threshold = thresholds[index]
sens = sensitivities[index]
spec = specificities[index]
print(
    f"Best threshold: {round(best_threshold,2)}. Sensitivity is"
    f" {round(sens*100,2)}% and Specificity is {round(spec*100,2)}% "
)

In [ ]:
# @title Show the results in a table
import pandas as pd

eval_embeddings_obj = eval_cancer_embeddings + eval_benign_embeddings

df = pd.DataFrame(
    {'ground_truth': eval_labels, 'model_score': test_predictions}
)
df['tumor_prediction'] = df['model_score'] > best_threshold
df['embeddings'] = [e.embedding for e in eval_embeddings_obj]

df

In [ ]:
import matplotlib.pyplot as plt


def render_patch_from_embedding(
    patch: gcs_image.GcsPatch, plot_name: str = ''
) -> None:
  patch_bytes = patch.image_bytes()
  plt.figure(figsize=(2, 2))
  plt.imshow(patch_bytes)
  plt.title(plot_name)
  plt.axis('off')
  plt.show()


# @title Visualize True Positives
def display_results(
    tumor_prediction: bool, ground_truth: int, title: str
) -> None:
  df_tp = (
      df[
          (df['tumor_prediction'] == tumor_prediction)
          & (df['ground_truth'] == ground_truth)
      ]
      .sort_values('model_score', ascending=False)
      .head(5)
  )
  for index, row in df_tp.iterrows():
    print(index)
    print(f'model score is {row.model_score}')
    render_patch_from_embedding(eval_embeddings_obj[index].patch, title)


display_results(True, 1, 'True Positive')


In [ ]:
# @title Visualize True Negatives
display_results(False, 0, 'True Negative')

In [ ]:
# @title Visualize False Positives
display_results(True, 0, 'False Positive')

In [ ]:
# @title Visualize False Negatives
display_results(False, 1, 'False Negative')

# Next steps

 Explore the other [notebooks](https://github.com/google-health/path-foundation/blob/master/notebooks).